In [1]:
from pyNBS import data_import_tools as dit
from pyNBS import network_propagation as prop
from pyNBS import pyNBS_core as core
from pyNBS import pyNBS_single
from pyNBS import consensus_clustering as cc
from pyNBS import pyNBS_plotting as plot

import time
import networkx as nx
import pandas as pd
import numpy as np
from IPython.display import Image

# Load Data

#### Load non-default parameters
Notes about the parameter file:
 - If no parameter path is given, default parameters will be set instead (see documentation for details and default values). 
 - The parameter file is a 2-column comma-separated text file where the first column is the parameter name, and the second column is the parameter value. The delimiter for this file must be a comma. 
 - Blank lines and lines starting with "#" will be ignored.
 - The parameter file may include as many or as few of the parameters from the pyNBS overall parameter space. For examples of two parameter files see: ```./OV_run_pyNBS_Hofree_params.csv``` VS ```./run_pyNBS_default_params.csv```
 
 
An excerpt of the the default parameters file is given below:  
```
################################  
#   Overall pyNBS parameters   #  
################################  
verbose,True  
outdir,./Results/  
  
###############################  
#   Data Loading Parameters   #  
###############################  
net_filedelim,"	"  
mut_filetype,matrix  
mut_filedelim,","  
degree_preserved_shuffle,False  
node_label_shuffle,False  
```

In [2]:
pyNBS_params = dit.load_params(params_file='./OV_run_pyNBS_Hofree_params.csv')

#### Load molecular network
The network file is a 2-column text file representing an unweighted network. Each row represents a single edge in the molecular network.    
  
Notes about the network file:  
 - The default column delimiter is a tab character '\t' but a different delimiter can be defined by the user here or in the parameter file with the "net_filedelim" parameter.
 - The network must not contain duplicate edges (e.g. TP53\tMDM2 is equivalent to MDM2\tTP53)
 - The network must not contain self-edges (e.g. TP53\tTP53)
 - Only the first two columns of a network file are read as edges for the network, all other columns will be ignored.
 - The load_network function also includes options to read in edge- or label-shuffled versions of the network, but by default, these options are turned off.
 
An excerpt of the first five rows of the PID network file is given below:  
```
A1BG	A2M
A1BG	AKT1
A1BG	GRB2
A1BG	PIK3CA
A1BG	PIK3R1
```

In [3]:
# The only required parameter here is the network file path, it may be defined by the user in the parameter file
# or explicitly here.
network_filepath = pyNBS_params['network_file']
network = dit.load_network_file(network_filepath, params=pyNBS_params)

Network File Loaded: ./Example_Notebook_Data/Network_Files/HM90.sif


#### Load binary somatic mutation data
The binary somatic mutation data file can be represented in two file formats:  
The ```matrix``` binary somatic mutation data format. This file format is a binary csv or tsv matrix with rows represent samples/patients and columns represent genes.  The following table is a small excerpt of a matrix somatic mutation data file:  

||A1CF|A2BP1|A2M|
|-|-|-|-|
|TCGA-04-1638|0|0|1|
|TCGA-23-1029|1|0|0|
|TCGA-23-2647|0|1|0|
|TCGA-24-1847|0|0|1|
|TCGA-42-2589|1|0|0|

The ```list``` binary somatic mutation data format. This file format is a 2-column csv or tsv list where the 1st column is a sample/patient and the 2nd column is a gene mutated in the sample/patient. There are no headers in this file format. Loading data with the list format is typically faster than loading data from the matrix format.The following text is the list representation of the matrix above.
```
TCGA-04-1638	A2M
TCGA-23-1029	A1CF
TCGA-23-2647	A2BP1
TCGA-24-1847	A2M
TCGA-42-2589	A1CF
```

In [4]:
# The only required parameter here is the somatic mutation data file path, it may be defined by the user in the 
# parameter file or explicitly here.
sm_data_filepath = pyNBS_params['sm_data_file']
sm_mat = dit.load_binary_mutation_data(sm_data_filepath, params=pyNBS_params)

Binary Mutation Matrix Loaded: ./Example_Notebook_Data/Mutation_Files/OV_sm_mat_Hofree.csv


# Construct regularization graph for use in network-regularized NMF
In this step, we will construct the graph used in the network-regularized non-negative matrix factorization (netNMF) step of pyNBS. This network is a K-nearest neighbor (KNN) network constructed from the network influence matrix (Vandin et al 2011*) of the molecular network being used to stratify tumor samples. The graph laplacian of this KNN network (knnGlap) is used as the regularizer in the following netNMF steps. This step uses the `network_inf_KNN_glap()` function in the pyNBS_core module. The function constructs the knnGlap with the following steps:  
  
__Steps to construct knnGlap:__
1. Construct the network influence matrix as described by Vandin et al 2011*
> i. Construct laplacian matrix of the molecular network.  
> ii. Adjust diagonal of the laplacian matrix by small gamma factor (default 0.01)  
> iii. Calculate the inverse of the diagonal-adjusted graph laplacian from (ii) to get the network influence matrix <br> Note: _This method is significantly faster and gives similar results as the original method used previously in Hofree's NBS v0.2.0, which calculated the pseudoinverse of each network component._
2. Construct KNN graph by conneting each node to its k nearest neighbors by influence score (default k is 11)
3. Calculate graph laplacian of this new KNN graph and return it as knnGlap

---
* Fabio Vandin, Eli Upfal, and Benjamin J. Raphael. Journal of Computational Biology. March 2011, 18(3): 507-522. https://doi.org/10.1089/cmb.2010.0265

In [ ]:
# Constructing knnGlap
knnGlap = core.network_inf_KNN_glap(network)

# Construct network propagation kernel matrix
Due to the multiple subsampling and propagation steps used in pyNBS, we have found that the algorithm can be significantly sped up for large numbers of subsampling and propagation iterations if a network propagation kernel can be pre-computed. Here we compute this propagation kernel by propagating the all genes in the molecular network independently of one another. The propagation profile of each tumor is then simply the row sums of all genes marked as mutated in each tumor, rather than having to perform the full network propagation step again after each subsampling of the data. Re-propagating at each subsampling step can be time consuming due to the matrix inversion calculation required by our implementation of the network propagation algorithm, which is based on the closed form of the random walk model over networks presented by the HotNet2 paper (Leiserson et al 2015*). This step uses the `network_propagation()` function in the network_propagation module of pyNBS.<br>

The general formulation of the closed form random-walk propagation for a given network: <br>
$$F_t = (1-\alpha)*F_0*(I-\alpha*A)^{-1}$$<br>
$F_0$ here is typically the binary mutation matrix, but for the kernel, it is the same as the identity matrix. A is the normalized adjcency matrix of the molecular network. The normalized adjacency matrix is calculated as $A*D^{-1}$ where D is the row/column-matched diagonalized node degree of the molecular network. $\alpha$ is the network propagation constant, a default value of 0.7 is used here (same as by Hofree), but results may vary if $\alpha$ is changed. Analysis by Hofree et al suggest that setting $\alpha$ between 0.5-0.8 gives relatively stable results.
  
Additional notes about how we perform random-walk based network propagation:  
 - We first separate the molecular network into each connected component and then perform network propagation for each connected component and concatenate the resulting kernel matrices along the diagonal for each subgraph.
 - Our default normalized adjacency matrix is not symmetric, but an option is available to calculate a symmetric degree-normalized adjacency matrix defined as $D^{-0.5} * A * D^{-0.5}$.  
 
---
* Leiserson MDM, Vandin F, Wu H-T, et al. Pan-Cancer Network Analysis Identifies Combinations of Rare Somatic Mutations across Pathways and Protein Complexes. Nature genetics. 2015;47(2):106-114. doi:10.1038/ng.3168.

In [ ]:
# Calculate propagation kernel by propagating identity matrix of network
network_nodes = network.nodes()
network_I = pd.DataFrame(np.identity(len(network_nodes)), index=network_nodes, columns=network_nodes)
kernel = prop.network_propagation(network, network_I, params=pyNBS_params)
print 'Network kernel calculated'

Performing network propagation with alpha: 0.7


# Subsampling, propagation, and netNMF
After the pre-computation of the regularization graph laplacian and the network propagation kernel, we perform the core steps of the NBS algorithm multiple times (100x here) to produce multiple patient clusters that will be used in the later consensus clustering step. Each patient clustering is performed with the following steps:  
  
__1. Subsample binary somatic mutation data__
>  i. Reduce binary somatic mutation data matrix to only contain columns of genes found in the network.  
>  ii. Sub-sample rows (samples/tumors) and columns (network genes) of the binary somatic mutation matrix. Default is 80% of each axis. This can be changed with the *pats\_subsample\_p* and *gene\_subsample\_p* value in the parameter file/dictionary.  
>  iii. Filter all rows with less than the minimum number of mutations. Default is 10 mutations. This can be changed with the *min\_muts* value in the parameter file/dictionary.  
  
__2. Propagate binary somatic mutation data over network__
>  i. If no network propagation kernel is pre-computed/provided, use the closed form of the random walk model over the full network by subgraphs (HotNet2)  
> ii. If a pre-calculated kernel is provided, calculate individual propagation profiles by calculating the column sums of all genes mutated in the patient for each patient. This method saves a significant amount of time when performing many iterations of these steps.  
  
__3. Quantile normalize the network-smoothed mutation data__
>  i. Sort each patient by gene propagation value  
>  ii. Rank averages for each gene across samples  
>  iii. Assign ranked averages to ranks of each gene for each sample  
  
__4. Use netNMF to decompose network data into k clusters__
>  i. In each iteration, update W and H with network constraints until W and H converge  
> ii. Track reconstruction errors and residuals  
  
These steps are wrapped by the `pyNBS_single()` function, which calls each step above as a different function from the `pyNBS_core module` (with the exception of the network propagation step).

In [ ]:
Hlist = []
for i in range(pyNBS_params['niter']):
    netNMF_time = time.time()
    pyNBS_params['pyNBS_iteration'] = repr(i+1)
    pyNBS_params['verbose'] = False # Turn off reporting for now at this stage
    Hlist.append(pyNBS_single.NBS_single(sm_mat, propNet=network, propNet_kernel=kernel, regNet_glap=knnGlap, params=pyNBS_params))
    if pyNBS_params['verbose']:
        t = time.time()-netNMF_time
        print 'NBS iteration:', i+1, 'complete:', t, 'seconds'

# Consensus Clustering
In order to produce robust patient clusters, the sub-sampling and re-clustering steps as done above are needed. After the patient data is subsampled multiple times (default=100), we perform the following step on each individual clustering result to create a single consensus clustering of all patients.  
  
__Steps for consensus clustering__
1. Assign each patient to a cluster by the argmax column in the 'H' matrix for each netNMF result (termed 'hard cluster assignment')
2. Construct patient x patient co-clustering matrix where each element is the proportion of all clustering iterations where any two patients were assigned to the same cluster.
3. Transform this simiarity matrix into a distance matrix by taking 1-[co_clustering_matrix]
4. Apply hierarchical clustering to the resulting distance matrix from (3) by average linkage and cut the hierarchy at desired depth k (the number of clusters, default is 3) to assign patients to a consensus cluster.

This step uses the consensus_hclust_hard() function in the conensus_clustering module. It accepts a list of pandas dataframes as generated in the previous step. If the H matrices were generated separately and saved to a directory, the user will need to manually import those H matrices into a python list first before passing the list to the function below.

In [ ]:
############################################################
# ---------- NBS Consensus Clustering Functions ---------- #
############################################################

import os
import pandas as pd
import numpy as np
import scipy.spatial.distance as dist
import scipy.cluster.hierarchy as hclust

# Constructs Hlist object for consensus clustering functions if NBS iterations were run in parallel and outputs saved to a folder
def Hlist_constructor_from_folder(folder, ext='.csv', normalize_H=False, verbose=False):
    co_clustering_results = [folder+fn for fn in os.listdir(folder) if fn.endswith(ext)]
    # Generate list of patient clusterings from netNMF
    Hlist = [pd.read_csv(fn, index_col=0) for fn in co_clustering_results]
    # Normalize H matrices if needed (to make columns comparable if not already done in decomposition)
    if normalize_H:
        Hlist_norm = []
        for H in Hlist:
            H_norm = np.dot(H,np.diag(1/H.sum()))
            Hlist_norm.append(pd.DataFrame(H_norm, index=H.index))
        if verbose:
            print 'Hlist constructed and normalized'
        return Hlist_norm
    else:
        if verbose:
            print 'Hlist constructed'
        return Hlist

# Takes a list of 'H' (patient-by-k) dataframes and performs 'hard' consensus clustering
# Using hierarchical clustering and average linkage
# Returns similarity table (distance is 1-similarity) and linkage map of patients
# Also returns cluster assignment map of patients if wanted
def consensus_hclust_hard(Hlist, params=None):
    # Load and set consensus clustering parameters
    # Make sure all H matrices are pandas DataFrames
    if not all([type(H)==pd.DataFrame for H in Hlist]):
        raise ValueError('Not all H matrices given are pandas DataFrames.')
    k = Hlist[0].shape[1]
    hclust_linkage_method = 'average'
    hclust_linkage_metric = 'euclidian'
    save_cc_matrix = True
    save_clusters = True
    verbose = False
    if (params is not None) and (type(params)==dict):
        if 'netNMF_k' in params:
            k = int(params['netNMF_k'])         
        if 'hclust_linkage_method' in params:
            hclust_linkage_method = str(params['hclust_linkage_method'])   
        if 'hclust_linkage_metric' in params:
            hclust_linkage_metric = str(params['hclust_linkage_metric'])   
        if 'save_cc_matrix' in params:
            save_cc_matrix = bool(params['save_cc_matrix'])   
        if 'save_clusters' in params:
            save_clusters = bool(params['save_clusters'])   
        if 'verbose' in params:
            verbose = bool(params['verbose']) 
    # Generate patient list
    pat_list = set()
    for H in Hlist:
        pat_list = pat_list.union(set(H.index))
    pat_list = sorted(list(pat_list))
    if verbose:
        print 'Constructing Hlist:', len(Hlist), 'cluster matrices', len(pat_list), 'samples'

    # Initialzie co-clustering tables
    co_clust_table = pd.DataFrame(0, index=pat_list, columns=pat_list)
    cluster_count = pd.DataFrame(0, index=pat_list, columns=pat_list)

    # Calculate patient similarities and linkage
    for H in Hlist:
        H.columns = range(1,len(H.columns)+1)
        # Update patient cluster count
        cluster_count.ix[H.index, H.index]+=1
        # Get cluster assignment for each patient
        cluster_assign = {i:[] for i in H.columns}
        for pat in H.index:
            cluster_assign[np.argmax(H.ix[pat])].append(pat)
        # Update co-clustering matrix with each cluster assignment
        for cluster in cluster_assign:
            cluster_pats = cluster_assign[cluster]
            co_clust_table.ix[cluster_pats, cluster_pats]+=1
    cc_hard_sim_table = co_clust_table.astype(float).divide(cluster_count.astype(float)).fillna(0)
    cc_hard_dist_table = 1-cc_hard_sim_table
    Z = hclust.linkage(dist.squareform(np.array(cc_hard_dist_table)), method=hclust_linkage_method, metric=hclust_linkage_metric)
    cluster_map = hclust.fcluster(Z, k, criterion='maxclust')
    cluster_assign = pd.Series({cc_hard_dist_table.index[i]:cluster_map[i] for i in range(len(cc_hard_dist_table.index))}, name='CC Hard, k='+repr(k))
    if save_cc_matrix:
        save_cc_matrix_path = params['outdir']+params['job_name']+'_cc_matrix.csv'
        cc_hard_sim_table.to_csv(save_cc_matrix_path)
    if save_clusters:
        save_clusters_path = params['outdir']+params['job_name']+'_cluster_assignments.csv'
        cluster_assign.to_csv(save_clusters_path)        
    if verbose:
        print 'Hlist consensus constructed and sample clusters assigned'
    return cc_hard_sim_table, Z, cluster_assign

In [ ]:
pyNBS_params['verbose'] = True
NBS_cc_table, NBS_cc_linkage, NBS_cluster_assign = consensus_hclust_hard(Hlist, params=pyNBS_params)

# Co-Clustering Map
To visualize the clusters formed by the pyNBS algorithm, we can plot a similarity map using the objects created in the previous step. We will also load data from the original Hofree et al 2013 paper to compare the results of the pyNBS implementation of the algorithm to the results reported in the paper. This step uses the `cluster_color_assign()` and `plot_cc_map()` functions in the `pyNBS_plotting` module.

In [ ]:
# First load the cluster assignment data from Hofree 2013 for ovarian cancer patients
orig_Hofree_OV_clust = pd.read_table('./Example_Notebook_Data/Hofree_Results/Hofree_OV_NBS_Results.csv',sep=',',index_col=0)

# Align the pyNBS and Hofree cluster assignments with one another
cluster_align = pd.concat([orig_Hofree_OV_clust.iloc[:,0], NBS_cluster_assign], axis=1).dropna(axis=0,how='any').astype(int)
Hofree_OV_clust = cluster_align.iloc[:,0].astype(int)
pyNBS_OV_clust = cluster_align.iloc[:,1].astype(int)

In [ ]:
# Assign colors to clusters from Hofree and pyNBS
Hofree_OV_clust_cmap = plot.cluster_color_assign(Hofree_OV_clust, name='Hofree OV Cluster Assignments')
pyNBS_OV_clust_cmap = plot.cluster_color_assign(pyNBS_OV_clust, name='pyNBS OV Cluster Assignments')
# Plot and save co-cluster map figure
plot.plot_cc_map(NBS_cc_table, NBS_cc_linkage, row_color_map=Hofree_OV_clust_cmap, col_color_map=pyNBS_OV_clust_cmap, 
                 params=pyNBS_params)

In [ ]:
Image(filename = pyNBS_params['outdir']+pyNBS_params['job_name']+'_cc_map.png', width=600, height=600)

# Survival analysis
To determine if the patient clusters are prognostically relevant, we perform a standard survival analysis using a multi-class logrank test to evaluate the significance of survival separation between patient clusters. This data is plotted using a Kaplan-Meier plot using the `cluster_KMplot()` in the `pyNBS_plotting` module. However in order to plot the survival differences between clusters, we will need to load survival data for each patient. This data was extracted from TCGA clinical data. The survival data is given in a 4-column delimited table with the specific headings described below (the columns must be in the same order as shown below). 
  
__Survival Table Column Headers:__
 - `vital_status`: Binary (0/1) value of whether or not the patient is alive (censored). 0 for alive, 1 for dead.
 - `days_to_death`: If the patient has a death event in the `vital_status` column, this is the number of days the patient survived from diagonsis. Otherwise the value is 0 if the patient is still alive.
 - `days_to_last_followup`: If the patient is still alive, this the last time the patient was known to have followed up and was still alive. Otherwise the value is 0 if the patient is dead.
 - `overall_survival`: This is either the days until the patient's death or until their last follow-up. This is the max of the aforementioned previous 2 columns.

The following is an example of a few lines of the Hofree OV survival table:  

||vital_status|days_to_death|days_to_last_followup|overall_survival|
|-|-|-|-|-|
|TCGA-3P-A9WA|0|0|420|420|
|TCGA-59-A5PD|1|624|0|624|
|TCGA-5X-AA5U|0|0|361|361|
|TCGA-04-1331|1|1336|0|1336|
|TCGA-04-1332|1|1247|0|1247|

Note: The default setting for pyNBS is that no survival curves are drawn because the survival data is not a required parameter. The path to valid survival data must be explicitly defined.

In [ ]:
# Load survival Data
surv_data = './Example_Notebook_Data/Clinical_Files/OV.clin.merged.Hofree.txt'
# Plot KM Plot for patient clusters
cluster_KMplot(NBS_cluster_assign, surv_data, params=pyNBS_params)
Image(filename = pyNBS_params['outdir']+pyNBS_params['job_name']+'_KM_plot.png', width=600, height=600)

# pyNBS Result comparison to Hofree et al 2013
We also compare the pyNBS clustering results against the original Hofree 2013 cluster assignments of the same patient data using two scores: adjusted rand index and adjusted mutual information score.
  
__Adjusted Rand Index (ARI)__
> Rand index adjusted for chance.
$$ARI = \frac{RI - \mathbb{E}(RI)}{ \max(RI) - \mathbb{E}(RI)}$$

__Adjusted Mutual Info Score (AMI)__
> Adjusted Mutual Information between two clusterings.
$${AMI(U, V) = \frac{MI(U, V) - \mathbb{E}(MI(U, V))}{\max(H(U), H(V)) - \mathbb{E}(MI(U, V))}}$$

In [ ]:
from sklearn.metrics.cluster import adjusted_mutual_info_score, adjusted_rand_score

In [ ]:
adj_rand_index = adjusted_rand_score(Hofree_OV_clust, pyNBS_OV_clust)
adj_mutual_info_score = adjusted_mutual_info_score(Hofree_OV_clust, pyNBS_OV_clust)
print 'Adjusted Rand Index is: ' + str(adj_rand_index)
print 'Adjusted Mutual Info Score is: ' + str(adj_mutual_info_score)